**NDVI com Google Earth Engine (Python)**

Por: Sandy dos Reis Hermann

26/05/2025

**Análise do monitoramento de 5 anos da cobertura vegetal em área de restauração com uso de imagens de satélite: uma abordagem usando Google Earth Engine e Google Colab.**

- Esse projeto tem como objetivo analisar por meio das geotecnologias, a cobertura vegetal da área de restauro ao entorno da PCH Anhanguera, localizada no município de São Joaquim da Barra - SP -
- Este script utiliza o Google Earth Engine para calcular o NDVI (Índice de Vegetação por Diferença Normalizada) de imagens do satélite Sentinel-2 na área de interesse com raio de 5 km, para os anos de 2018 a 2022. Os resultados são exportados automaticamente para o Google Drive.

✅ Requisitos

- Conta no Google Earth Engine (https://signup.earthengine.google.com)
- Biblioteca Python 'earthengine-api' instalada
- Autenticação feita com ee.Authenticate() e ee.Initialize()
- Permissão para gravar no seu Google Drive



**Primeiro vamos criar seu proprio projeto no Google Earth Engine**

Antes de rodar o script, você precisa ter uma conta e um projeto no Google Earth Engine (GEE). Siga esses passos simples:

Crie sua conta no GEE - Acesse: https://earthengine.google.com/

* Clique em Sign Up e use sua conta Google para se cadastrar.
* Espere a aprovação (pode levar algumas horas ou dias).

Crie um projeto no Google Cloud

* Acesse: https://console.cloud.google.com/

* Clique em Selecionar projeto > Novo projeto.

* Dê um nome para o projeto (ex: meu-projeto-gee).

* Anote o nome do projeto, pois ele será usado no script.

* Ative a API do Earth Engine no projeto

Dentro do Google Cloud, vá em APIs e Serviços > Bibliotecas.

* Procure por Earth Engine API e clique em Ativar.

Pronto para usar!

Agora, quando rodar o script, **use o nome do seu projeto no lugar de projeto-teste-sr-459612**.

Primeiro vamos importar as bibliotecas e autenticar sua conta!

In [ ]:
import ee
import geemap
import ipywidgets as widgets

Nessa parte vamos entrar em seu projeto do gee

In [ ]:
# Autenticar e inicializar
ee.Authenticate()
ee.Initialize(project='projeto-teste-sr-459612')

**🌍 Área de estudo - São Joaquim da Barra**

O script define um ponto central:

Latitude: -20.51016  
Longitude: -47.85615

In [ ]:
# Localização
latitude = -20.51016
longitude = -47.85615
ponto = ee.Geometry.Point([longitude, latitude])
aoi_buffer = ponto.buffer(5000)

Agora vamos definir funções de tirar nuvens, calcular ndvi por ano, os parametros de visualização do mapa...

In [ ]:
# Função para mascarar nuvens
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Calcular NDVI
def calc_ndvi(img):
    return img.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Parâmetros de visualização
vis_ndvi = {'min': 0, 'max': 1, 'palette': ['white', 'green']}
vis_params=[vis_ndvi] * len(ndvi_images),

# Lista de anos
anos = list(range(2018, 2023))
ndvi_images = []

# Calcular NDVI médio por ano
for ano in anos:
    inicio = ee.Date(f'{ano}-01-01')
    fim = ee.Date(f'{ano}-12-31')

    colecao = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(inicio, fim) \
        .filterBounds(aoi_buffer) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds)

    ndvi = colecao.map(calc_ndvi).mean().clip(aoi_buffer)
    ndvi_images.append(ndvi)

# Criar mapas vinculados
geemap.linked_maps(
    rows=1,
    cols=5,
    height="400px",
    center=[latitude, longitude],
    zoom=11,
    ee_objects=ndvi_images,
    vis_params=[vis_ndvi] * len(ndvi_images),
    labels=[str(ano) for ano in anos],
    label_position="topright"
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

Agora vamos exportar essas imagens em uma pasta do drive

In [ ]:
# Exportar as imagens NDVI médias anuais para o Google Drive
for i, ano in enumerate(anos):
    tarefa = ee.batch.Export.image.toDrive(
        image=ndvi_images[i].toFloat(),
        description=f'NDVI_{ano}',
        folder='NDVI_exports',   # pasta no Drive, pode alterar ou deixar vazio para raiz
        fileNamePrefix=f'NDVI_{ano}',
        region=aoi_buffer.bounds().getInfo()['coordinates'],
        scale=10,                # resolução do Sentinel-2 (10m)
        crs='EPSG:4326',         # sistema de referência
        maxPixels=1e13
    )
    tarefa.start()
    print(f'Exportação do NDVI do ano {ano} iniciada.')